In [1]:
try:
    # Comment out if not using colab
    from google.colab import drive
    drive.mount('/content/drive')

    # Specific for luca's computer
    %cd "/content/drive/Othercomputers/lucas-yoga/Current/INFO381/code/INFO381-GitHub"
    using_colab = True
except:
    print("Not using Google Colab")
    using_colab = False

Mounted at /content/drive
/content/drive/Othercomputers/lucas-yoga/Current/INFO381/code/INFO381-GitHub


In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\lucaf\appdata\local\temp\pip-req-build-yf97rgxc
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\lucaf\AppData\Local\Temp\pip-req-build-yf97rgxc'


In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models
import torch.nn as nn
from sklearn.metrics import accuracy_score
import clip


from utils import get_dataloaders
from classifiers import CLIPBinaryClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
# Define resize transform for the CNN
resize_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

# Load both train and test dataloaders for the CNN model
cnn_train_loader, cnn_test_loader = get_dataloaders(
    zip_path="fake_vs_real.zip",
    batch_size=32,
    split='both',
    transform=resize_transform
)

# Setup the ResNet18 model
cnn_model = models.resnet18(pretrained=False)
num_ftrs = cnn_model.fc.in_features
cnn_model.fc = nn.Linear(num_ftrs, 2)
cnn_model.load_state_dict(torch.load("models/resnet18_cnn.pth"))

cnn_model.to(device)
cnn_model.eval()

# Define evaluation function (same as for CLIP)
def evaluate(model, dataloader):
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Evaluate on train and test sets
accuracy_train_cnn = evaluate(cnn_model, cnn_train_loader)
accuracy_test_cnn = evaluate(cnn_model, cnn_test_loader)

print(f"ResNet18 Train Accuracy: {accuracy_train_cnn:.2f}")
print(f"ResNet18 Test Accuracy: {accuracy_test_cnn:.2f}")


Running in Google Colab


KeyboardInterrupt: 

In [4]:
# # 1) Load the CLIP backbone and our classifier model
#clip_model, preprocess = clip.load("ViT-B/32", device=device)

# # Load both train and test dataloaders

# model = CLIPClassifier(clip_model, embed_dim=512, num_classes=2).to(device)
# model.load_state_dict(torch.load("models/clip_classifier_10epochs.pth", map_location=device))
# model.eval()

model = CLIPBinaryClassifier(model_path="models/clip_classifier_10epochs.pth", device=device)
clip_train_loader, clip_test_loader = get_dataloaders(zip_path="fake_vs_real.zip", batch_size=32, split='both', transform=model.preprocessor)

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 70.8MiB/s]


Running in Google Colab


In [5]:
# 2) Define an evaluation function
from sklearn.metrics import accuracy_score

def evaluate(model, dataloader):
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# 3) Evaluate on both sets
accuracy_train = evaluate(model, clip_train_loader)
accuracy_test = evaluate(model, clip_test_loader)

print(f"CLIP ViT Train Accuracy: {accuracy_train:.2f}")
print(f"CLIP ViT Test Accuracy: {accuracy_test:.2f}")


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


CLIP ViT Train Accuracy: 0.97
CLIP ViT Test Accuracy: 0.95
